# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> implement any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`bucket()`

`long_trips()`

`manhattan_trips()`

`weighted_profit()`

`final_output()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-info">
    Do <strong>not</strong> change the below cell. Run it to initialize your PySpark instance.
</div>

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1603303943864_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [29]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

from pyspark.sql.functions import col

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://fall2020-cse6242"
    
    # Load Trip Data
    trip_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trip_path, header=True, inferSchema=True)
    print("Trip Count: ",trips.count()) # Prints # of trips (# of records, as each record is one trip)
    
    # Load Lookup Data
    lookup_path = '/'+size+'/taxi*'
    lookup = spark.read.csv(input_bucket + lookup_path, header=True, inferSchema=True)
    
    return trips, lookup

def main(size, bucket):
    # Runs your functions implemented above.
    
    print(user())
    trips, lookup = load_data(size=size)
    trips = long_trips(trips)
    mtrips = manhattan_trips(trips, lookup)
    wp = weighted_profit(trips, mtrips)
    final = final_output(wp,lookup)
    
    # Outputs the results for you to visually see
    final.show()
    
    # Writes out as a CSV to your bucket.
    final.write.csv(bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function
</div>

## 3a. [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [3]:
def user():
    # Returns a string consisting of your GT username.
    return 'Zliu723'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3b. [2 pts] Update the `long_trips()` function
This function filters trips to keep only trips longer than 2 miles.

In [30]:
def long_trips(trips):
    # Returns a Dataframe with Schema the same as :trips:
    trips = trips.filter(col("trip_distance")>=2)
    return trips

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3c. [6 pts] Update the `manhattan_trips()` function

This function determines the top 20 locations with a `DOLocationID` in manhattan by passenger_count (pcount).

Example output formatting:

```
+--------------+--------+
| DOLocationID | pcount |
+--------------+--------+
|             5|      15|
|            16|      12| 
+--------------+--------+
```

In [31]:
# import pyspark.sql.functions as sf


def manhattan_trips(trips, lookup):
    # Returns a Dataframe with Schema: DOLocationID, pcount
#     df1 = trips.groupBy(col("DOLocationID")).agg({"passenger_count":"sum"}).withColumn('pcount',col("sum(passenger_count)"))
    
#     df2 = df1.join(lookup,col("DOLocationID") == col("LocationID")).filter(col("Borough") == "Manhattan").select(["DOLocationID","pcount"]).orderBy(col("pcount"),ascending =False).limit(20)
    
    df1 = trips.join(lookup,col("DOLocationID") == col("LocationID")).filter(col("Borough") == "Manhattan")
    df2 = df1.groupBy(col("DOLocationID")).agg({"passenger_count":"sum"}).withColumn('pcount',col("sum(passenger_count)"))
    df = df2.select(["DOLocationID","pcount"]).orderBy(col("pcount"),ascending =False).limit(20)
    
    return df



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------+
|DOLocationID|pcount|
+------------+------+
|         236|521569|
|         237|457669|
|         161|454203|
|         170|374803|
|         230|364871|
|         162|361011|
|         142|338505|
|          48|333614|
|         239|323879|
|         234|316533|
|         141|315951|
|         186|302751|
|         238|277609|
|         163|276887|
|          79|267789|
|          68|262231|
|         107|253591|
|         263|249380|
|         164|244210|
|         140|235890|
+------------+------+

## 3d. [6 pts] Update the `weighted_profit()` function
This function should determine the average `total_amount`, the total count of trips, and the total count of trips ending in the top 20 destinations and return the `weighted_profit` as discussed in the homework document.

Example output formatting:
```
+--------------+-------------------+
| PULocationID |  weighted_profit  |
+--------------+-------------------+
|            18| 33.784444421924436| 
|            12| 21.124577637149223| 
+--------------+-------------------+
```

In [71]:
def weighted_profit(trips, mtrips): 
    # Returns a Dataframe with Schema: PULocationID, weighted_profit
    
    #step 1: df1 computes "average total amount" and "total count of all trips"
    df1 = trips.groupBy(col("PULocationID")).agg({"total_amount":"avg","PULocationID":"count"})

    #step 2: df2 contains all details with only 20 top DOL locations 
    mtrips = mtrips.withColumn("DOLocationID2",col("DOLocationID")).drop(col("DOLocationID"))
    df2 = trips.join(mtrips,col("DOLocationID") == col("DOLocationID2")).withColumn('PULocationID2',col("PULocationID")).select(["PULocationID2","DOLocationID2"])
    
    #step 3: df3 includes the df1 columns, df3_temp computes new column "total count of trips ending in the top 20 destinations"
    df3 = df1.join(df2,col("PULocationID")==col("PULocationID2"))
    df3_temp = df3.groupBy(col('PULocationID2')).agg({"PULocationID2":"count"})
    
    #step 4: add df3_temp's new column into the main dataframe, computer new column "weighted_profit"
    df4 = df3.drop(col('PULocationID2'))
    df4 = df4.join(df3_temp,col("PULocationID")==col("PULocationID2")).withColumn("weighted_profit",col('count(PULocationID2)')/col("count(PULocationID)")*col("avg(total_amount)")) \
             .orderBy(col('PULocationID')) \
            .select(["PULocationID","weighted_profit"]) 
    
    #step 5: remove distinct value
    df5 = df4.distinct()
  
    return df5



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 3e. [5 pts] Update the `final_output()` function
This function will take the results of `weighted_profit`, links it to the `borough` and `zone` and returns the top 20 locations with the highest `weighted_profit`.

Example output formatting:
```
+------------+---------+-------------------+
|    Zone    | Borough |  weighted_profit  |
+----------------------+-------------------+
| JFK Airport|   Queens|  16.95897820117925|
|     Jamaica|   Queens| 14.879835188762488|
+------------+---------+-------------------+
```

In [65]:
def final_output(calc, lookup): 
    # Returns a Dataframe with Schema: Zone, Borough, weighted_profit
    df1 = calc.join(lookup,col("LocationID") == col("PULocationID")).select(["Zone","Borough","weighted_profit"]).orderBy(col("weighted_profit"),ascending = False).limit(20)
    
    return df1



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+------------------+
|                Zone|      Borough|   weighted_profit|
+--------------------+-------------+------------------+
|        Baisley Park|       Queens|21.060544583116098|
|     Randalls Island|    Manhattan|20.434121515335207|
|       South Jamaica|       Queens| 19.80635321457101|
|Flushing Meadows-...|       Queens|19.525570123421407|
|         Jamaica Bay|       Queens|           18.5028|
|   LaGuardia Airport|       Queens|18.175503156231095|
|       Arden Heights|Staten Island|17.625600000000002|
|         JFK Airport|       Queens|17.047973108292663|
|Saint George/New ...|Staten Island|  16.2431660899654|
|        Bloomingdale|    Manhattan|14.711553041789287|
|  Murray Hill-Queens|       Queens|14.345707821376667|
|     Mariners Harbor|Staten Island|   13.861982421875|
|             Jamaica|       Queens|12.427534360693343|
|       Rikers Island|        Bronx|           12.3675|
|             Maspeth|       Queens|11.908022338

<div class="alert alert-block alert-info">
    Test your code on the small dataset first, as the large dataset will take a significantly longer time to run
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> use the same bucket url for multiple runs of the `main()` function, as this will cause errors. Make sure to change the name of your output location every time. (ie: s3://cse6242-gburdell3/output-small2)
</div>

Update the below cell with the address to your bucket, then run the below cell to run your code to store the results in S3.

When you have confirmed the results of the small dataset, run it again using the large dataset. Your output file will appear ina folder in your s3 bucket called YOUROUTPUT.csv as a csv file with a name something like part-0000-4d992f7a-0ad3-48f8-8c72-0022984e4b50-c000.csv. Download this file and rename it to q3_output.csv for submission. Do not make any other changes to the file. 

In [68]:
bucket = 's3://cse6242-zliu723/output_large'

#main('small',bucket)
main('large', bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Zliu723
Trip Count:  187203269
+--------------------+-------------+------------------+
|                Zone|      Borough|   weighted_profit|
+--------------------+-------------+------------------+
|        Baisley Park|       Queens|29.360455779130838|
|Flushing Meadows-...|       Queens|27.304845733617668|
|       South Jamaica|       Queens|26.294916239873483|
|     Randalls Island|    Manhattan| 24.15098994022752|
|        Astoria Park|       Queens|21.706417112147513|
|Briarwood/Jamaica...|       Queens|19.945064631789332|
|Springfield Garde...|       Queens|19.468309288781903|
|             Jamaica|       Queens|19.283943000137892|
|              Corona|       Queens|18.228769248155974|
|   LaGuardia Airport|       Queens|18.181338808373003|
|         Jamaica Bay|       Queens|  17.1005294467579|
|             Maspeth|       Queens|17.005450640079548|
|Eltingville/Annad...|Staten Island| 16.83776475694445|
|         JFK Airport|       Queens|16.777725348249636|
|        Battery 

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be run or used when grading
</div>

In [72]:
df = load_data(size='small')


df[0].printSchema()  #the trips dataframe
df[1].printSchema() #the lookup dataframe


q3b= long_trips(df[0])
q3b.show(5)

q3c =manhattan_trips(df[0], df[1])
q3c.show(20)


q3d =weighted_profit(df[0], q3c)
q3d.show(10)


q3e =final_output(q3d,df[1])
q3e.show(20)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Trip Count:  7667792
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: stri